<a href="https://colab.research.google.com/github/kdpbiiitd/CSE508_Winter2023_A2_39./blob/main/IR_A2_Q2_IF_TDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import string
import nltk
import math
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [10]:
# Load the data
df = pd.read_csv('dataset.csv')

# Clean the text data
stopwords = nltk.corpus.stopwords.words('english')
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = " ".join([word.lower() for word in text.split() if word.lower() not in stopwords])
    return text

df['Text'] = df['Text'].apply(clean_text)

In [11]:
# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

# Calculate the prior probabilities of each category in the training set
prior_probs = defaultdict(float)
total_count = len(train_df)
for category, count in train_df['Category'].value_counts().items():
    prior_probs[category] = count / total_count

In [13]:
# Calculate the TF-IDF values for each word in each category
tfidf = TfidfVectorizer()
tfidf.fit(train_df['Text'])
tfidf_values = tfidf.transform(train_df['Text'])

tfidf_dict = defaultdict(dict)
for i, category in enumerate(train_df['Category']):
    for j, word in enumerate(tfidf.get_feature_names_out()):
        tfidf_dict[category][word] = tfidf_values[i, j]

#create a dictionary to store the likelihood probabilities
likelihood_probs = defaultdict(dict)
for label, counter in tfidf_dict.items():
    total_count = sum(counter.values())
    for word, count in counter.items():
        likelihood_probs[label][word] = count / total_count

In [14]:
# Calculate the posterior probabilities for each category for each document in the test set
posterior_probs = defaultdict(dict)
for i, row in test_df.iterrows():
    for category, prior_prob in prior_probs.items():
        posterior_prob = prior_prob
        for word in row['Text'].split():
            if word in likelihood_probs[category]:
                posterior_prob *= likelihood_probs[category][word]
        posterior_probs[row['Category']][category] = posterior_prob

In [15]:
#Use the naive bayes classifier to predict the category of each document in the test set
predictions = []
for i, row in test_df.iterrows():
    max_prob = -1
    max_category = None
    for category, posterior_prob in posterior_probs[row['Category']].items():
        if posterior_prob > max_prob:
            max_prob = posterior_prob
            max_category = category
    predictions.append(max_category)


In [16]:
# Calculate the accuracy, precision, recall, and F1 score of the classifier
print('Accuracy: ', accuracy_score(test_df['Category'], predictions))
print('Precision: ', precision_score(test_df['Category'], predictions, average='weighted'))
print('Recall: ', recall_score(test_df['Category'], predictions, average='weighted'))
print('F1 Score: ', f1_score(test_df['Category'], predictions, average='weighted'))

Accuracy:  0.22595078299776286
Precision:  0.05105375633730212
Recall:  0.22595078299776286
F1 Score:  0.08328842730939433


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
# Prompt the user to enter a document number
doc_num = int(input("Enter the document number: "))

# Get the predicted category and true category for the document
predicted_category = print(predictions[doc_num])
true_category = print(test_df.iloc[doc_num]['Category'])

Enter the document number: 45
sport
business
